In [33]:
import json
from pprint import pprint

import diskcache as dc
import requests
import jsonlines 

from deepl import Translator
from pydantic import BaseModel
from tqdm import tqdm

from config import TLCPaths
from data_loading import load_tlc_samples, get_annotation_ids
from models import Match, SampleCollection, Annotation, ProdigySample, ProdigyNERLabel

In [34]:
sample_collection = SampleCollection(load_tlc_samples())
annotations = [ann for sample in load_tlc_samples() for ann in sample.annotations]

def get_annotation_by_id(id):
    for ann in annotations:
        if ann.id == id:
            return ann
    return None

with open(TLCPaths.project_data_path.joinpath('search_terms_single_and_ids_strict_uniqueness.json'), 'r') as fp:
    terms_and_ids = json.load(fp)
    


In [35]:
annotations_file_path = TLCPaths.project_data_path.joinpath("samples_to_annotate.json")
validations_file_path = TLCPaths.project_data_path.joinpath("samples_to_validate.json")
# load the two json files 
with open(annotations_file_path, "r") as f:
    annotation_samples = [x for x in json.load(f)]
with open(validations_file_path, "r") as f:
    validation_samples = [x for x in json.load(f)]

print(f"{len(annotation_samples)=}")
print(f"{len(validation_samples)=}")


len(annotation_samples)=326
len(validation_samples)=672
len(annotation_samples)=326
len(validation_samples)=672


In [36]:
validation_annotations = []
validation_cuis = {}
for validation_sample in validation_samples:
    stem = validation_sample["stem"]
    ids = terms_and_ids[stem.strip()]
    annotation = get_annotation_by_id(ids[0])
    validation_annotations.append(annotation)
    validation_cuis[annotation.id] = validation_sample["cui"]

In [37]:
annotation_annotations = []
for annotation_sample in annotation_samples:
    stem = annotation_sample["stem"]
    ids = terms_and_ids[stem.strip()]
    annotation = get_annotation_by_id(ids[0])
    annotation_annotations.append(annotation)

In [38]:
# translate mentions and synonym to english
# use mention and synonym to find cui. if disagree take mention if tech term or take synonym is lay term
# query umls for german description. if not available take english one and translate back to german

In [39]:
class TranslatedTerm(BaseModel):
    name: str
    cui: str = ""


class TranslatedAnnotation(BaseModel):
    annotation: Annotation
    translated_mention: TranslatedTerm
    translated_synonym: TranslatedTerm


class CachedTranslator:
    def __init__(self, cache, api_key):
        self.cache = cache
        self.translator = Translator(api_key)

    def translate_text(self, text, source_lang, target_lang):
        if (text, source_lang, target_lang) in self.cache:
            return self.cache[(text, source_lang, target_lang)]
        else:
            res = self.translator.translate_text(text, source_lang=source_lang,
                                                 target_lang=target_lang)
            self.cache[(text, source_lang, target_lang)] = res
            return res


deepl_cache = dc.Cache("caches/deepl_cache")
deepl_key = "0c25ea6d-b79f-288a-541f-ae25709c6312:fx"
translator = CachedTranslator(deepl_cache, deepl_key)

In [40]:
res = translator.translate_text("Krankenhausmauer", source_lang="DE", target_lang="EN-US")
res.text

'Hospital Wall'

'Hospital Wall'

In [41]:
translated_annotations = []
for annotation in tqdm(annotation_annotations):
    mention = annotation.get_mention()
    translated_mention = translator.translate_text(mention.lower(), source_lang="DE",
                                                   target_lang="EN-US").text
    if len(annotation.synonyms) == 0:
        translated_synonym = ""
    else:
        synonym = annotation.synonyms[0]
        translated_synonym = translator.translate_text(synonym.lower(), source_lang="DE",
                                                       target_lang="EN-US").text
    translated_annotations.append(TranslatedAnnotation(annotation=annotation,
                                                       translated_mention=TranslatedTerm(
                                                           name=translated_mention),
                                                       translated_synonym=TranslatedTerm(
                                                           name=translated_synonym)))

100%|██████████| 326/326 [00:00<00:00, 4901.19it/s]


In [42]:
translated_annotations[-1]

TranslatedAnnotation(annotation=Annotation(tech_term=None, lay_term='Harnröhreninfekt', type=<TermType.LAY: 'LAY'>, span_start=927, span_end=943, synonyms=['Bakterielle Urethritis', 'Bakterielle Urethritis'], id=2742), translated_mention=TranslatedTerm(name='urethral infection', cui=''), translated_synonym=TranslatedTerm(name='bacterial urethritis', cui=''))

TranslatedAnnotation(annotation=Annotation(tech_term=None, lay_term='Harnröhreninfekt', type=<TermType.LAY: 'LAY'>, span_start=927, span_end=943, synonyms=['Bakterielle Urethritis', 'Bakterielle Urethritis'], id=2742), translated_mention=TranslatedTerm(name='urethral infection', cui=''), translated_synonym=TranslatedTerm(name='bacterial urethritis', cui=''))

In [43]:
UMLS_KEY = "43f9234c-4977-45f6-a440-2dda1b43d919"

umls_cache = dc.Cache("caches/umls_cache")


@umls_cache.memoize()
def get_cui(name):
    query_url = f"https://uts-ws.nlm.nih.gov/rest/search/current?apiKey={UMLS_KEY}&string={name}&searchType=normalizedString"
    ### send a get request to query url and get response
    response = requests.get(query_url)
    if not "result" in response.json():
        print("name: ", name, response.json())
        return ""
    results = response.json()["result"]["results"]
    if results:
        return results[0]["ui"]
    else:
        return ""


cui = get_cui(name="Pyelonephritis")
print(cui)

C0034186
C0034186


In [44]:
for translated_annotation in tqdm(translated_annotations):
    if translated_annotation.translated_mention.cui == "":
        translated_annotation.translated_mention.cui = get_cui(
            name=translated_annotation.translated_mention.name)
    if translated_annotation.translated_synonym.cui == "":
        translated_annotation.translated_synonym.cui = get_cui(
            name=translated_annotation.translated_synonym.name)


100%|██████████| 326/326 [00:00<00:00, 7779.43it/s]


In [45]:
# annotations_with_match = []  # entry = (annotation, translated_name, cui)
# annotation_without_match = []  # entry = (annotation)    
# for annotation in tqdm(translated_annotations):
#     if annotation.translated_synonym.cui and not annotation.translated_mention.cui:
#         entry = (annotation.annotation, annotation.translated_synonym.name,
#                  annotation.translated_synonym.cui)
#     elif not annotation.translated_synonym.cui and annotation.translated_mention.cui:
#         entry = (annotation.annotation, annotation.translated_mention.name,
#                  annotation.translated_mention.cui)
#     elif not annotation.translated_synonym.cui and not annotation.translated_mention.cui:
#         annotation_without_match.append(annotation)
#         continue
#     else:
#         if annotation.annotation.type == "TECH":
#             entry = (annotation.annotation, annotation.translated_mention.name,
#                      annotation.translated_mention.cui)
#         else:
#             entry = (annotation.annotation, annotation.translated_synonym.name,
#                      annotation.translated_synonym.cui)
#     annotations_with_match.append(entry)
annotations_with_match = []
annotations_without_match = []

for annotation in tqdm(translated_annotations):
    has_synonym_cui = bool(annotation.translated_synonym.cui)
    has_mention_cui = bool(annotation.translated_mention.cui)
    
    if has_synonym_cui and not has_mention_cui:
        matched_term = annotation.translated_synonym
    elif not has_synonym_cui and has_mention_cui:
        matched_term = annotation.translated_mention
    elif not has_synonym_cui and not has_mention_cui:
        annotations_without_match.append(annotation)
        continue
    else:  # Both synonym and mention have CUIs
        matched_term = (
            annotation.translated_mention
            if annotation.annotation.type == "TECH"
            else annotation.translated_synonym
        )

    entry = (annotation.annotation, matched_term.name, matched_term.cui)
    annotations_with_match.append(entry)


100%|██████████| 326/326 [00:00<00:00, 952852.34it/s]


In [46]:
# found cuis for 203 annotations and not for 123 annotations

In [47]:
print(len(annotations_with_match))
print(annotations_with_match[:2])

203
[(Annotation(tech_term='Gallensteine', lay_term=None, type=<TermType.TECH: 'TECH'>, span_start=121, span_end=133, synonyms=['Cholelith', 'Cholelith'], id=6202), 'gallstones', 'C0008350'), (Annotation(tech_term=None, lay_term='Magen Darm Grippe', type=<TermType.LAY: 'LAY'>, span_start=9, span_end=26, synonyms=['Norovirus-Gastroenteritis', 'Norovirus-Gastroenteritis'], id=5687), 'norovirus gastroenteritis', 'C2242683')]
203
[(Annotation(tech_term='Gallensteine', lay_term=None, type=<TermType.TECH: 'TECH'>, span_start=121, span_end=133, synonyms=['Cholelith', 'Cholelith'], id=6202), 'gallstones', 'C0008350'), (Annotation(tech_term=None, lay_term='Magen Darm Grippe', type=<TermType.LAY: 'LAY'>, span_start=9, span_end=26, synonyms=['Norovirus-Gastroenteritis', 'Norovirus-Gastroenteritis'], id=5687), 'norovirus gastroenteritis', 'C2242683')]


In [48]:
print(len(annotation_without_match))  # no cui for mention or synonym found
print(annotation_without_match[0])

123
annotation=Annotation(tech_term='Defiblirator', lay_term=None, type=<TermType.TECH: 'TECH'>, span_start=1163, span_end=1175, synonyms=['Schockgeber', 'Schockgeber'], id=811) translated_mention=TranslatedTerm(name='defiblirator', cui='') translated_synonym=TranslatedTerm(name='shock generator', cui='')
123
annotation=Annotation(tech_term='Defiblirator', lay_term=None, type=<TermType.TECH: 'TECH'>, span_start=1163, span_end=1175, synonyms=['Schockgeber', 'Schockgeber'], id=811) translated_mention=TranslatedTerm(name='defiblirator', cui='') translated_synonym=TranslatedTerm(name='shock generator', cui='')


In [49]:
for annotation in annotation_without_match[:5]:
    print(annotation.annotation.get_mention(), annotation.translated_mention)
    if annotation.annotation.synonyms:
        print(annotation.annotation.synonyms[0], annotation.translated_synonym)
    print()

Defiblirator name='defiblirator' cui=''
Schockgeber name='shock generator' cui=''

stopfende name='stuffing' cui=''
obstipierend name='obstipating' cui=''

Teufelskreis name='vicious circle' cui=''
Circulus vitiosus name='circulus vitiosus' cui=''

schlimme Rückenschmerzen in der Höhe der Nieren name='bad back pain at the level of the kidneys' cui=''
Symptome des oberen Urogenitaltraktes name='symptoms of the upper urogenital tract' cui=''

Schmerzen an den Nieren und 3 fach erhöhte Entzündungswerte im Urin name='pain in the kidneys and 3 times increased inflammation values in the urine' cui=''
Infektion des oberen Urogenitaltraktes name='infection of the upper urogenital tract' cui=''

Defiblirator name='defiblirator' cui=''
Schockgeber name='shock generator' cui=''

stopfende name='stuffing' cui=''
obstipierend name='obstipating' cui=''

Teufelskreis name='vicious circle' cui=''
Circulus vitiosus name='circulus vitiosus' cui=''

schlimme Rückenschmerzen in der Höhe der Nieren name='b

In [50]:
import csv

with open("to_annotate_strict.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["mention", "synonym", "cui"])
    for annotation in annotation_without_match:
        writer.writerow([annotation.annotation.get_mention(), annotation.annotation.synonyms, ""])

In [51]:
filtered_anns = []
for ann in annotation_without_match:
    if ann.annotation.get_mention() == "Teufelskreis":
        print(ann.annotation)
        continue
    filtered_anns.append(ann)

tech_term=None lay_term='Teufelskreis' type=<TermType.LAY: 'LAY'> span_start=753 span_end=765 synonyms=['Circulus vitiosus', 'Circulus vitiosus'] id=3898
tech_term=None lay_term='Teufelskreis' type=<TermType.LAY: 'LAY'> span_start=753 span_end=765 synonyms=['Circulus vitiosus', 'Circulus vitiosus'] id=3898


In [52]:
# annotation_without_match = filtered_anns

In [53]:
# ## read in annotations from csv
# with open("to_annotate.copy.csv", "r") as f:
#     reader = csv.reader(f)
#     next(reader)
#     manual_annotations = []
#     for annotation, row in zip(annotation_without_match, reader):
#         mention = row[0]
#         synonym = row[1]
#         cui = row[2]
#         assert annotation.annotation.get_mention() == mention
#         manual_annotations.append((annotation, cui))
#         
# manual_annotations[0]

In [54]:
manual_annotations = []

In [55]:
all_annotations = []
for annotation in tqdm(annotations_with_match):
    all_annotations.append(dict(annotation=annotation[0], cui=annotation[2]))
for annotation in tqdm(manual_annotations):
    all_annotations.append(dict(annotation=annotation[0].annotation, cui=annotation[1]))
for annotation in tqdm(validation_annotations):
    all_annotations.append(
        dict(annotation=annotation, cui=validation_cuis[annotation.id][0]))

100%|██████████| 203/203 [00:00<00:00, 687202.35it/s]
0it [00:00, ?it/s]
100%|██████████| 203/203 [00:00<00:00, 1017256.53it/s]
0it [00:00, ?it/s]
100%|██████████| 672/672 [00:00<00:00, 443590.22it/s]


In [56]:
print(len(all_annotations)) # 123 missing from manual annotation

875
875


# add descriptions

In [57]:
umls_cache = dc.Cache("caches/umls_cache")

@umls_cache.memoize()
def get_cui_name(cui):
    query_url = f"https://uts-ws.nlm.nih.gov/rest/content/current/CUI/{cui}?apiKey={UMLS_KEY}"
    response = requests.get(query_url)
    if not "result" in response.json():
        print("returning none for ", cui)
        return None
    else:
        eng_name = response.json()["result"]["name"]
        name = translator.translate_text(text=eng_name, source_lang="EN", target_lang="DE").text
        return name
    
@umls_cache.memoize()
def get_description(cui):
    name = get_cui_name(cui=cui)
    
    ger_sabs = ["DMDICD10", "DMDUMD", "WHOGER", "ICPCGER", "LNC-DE-AT", "LNC-DE-DE", "MDRGER",
                "MSHGER"]
    eng_sabs = ["HPO", "MDR", "MSH", "SNOMEDCT_US"]

    query_url = f"https://uts-ws.nlm.nih.gov/rest/content/current/CUI/{cui}/definitions?apiKey={UMLS_KEY}"
    response = requests.get(query_url)
    if "result" in response.json():
        result = response.json()["result"]
        for res in result:
            if res["rootSource"] in ger_sabs:
                return f"{name}: " + res["value"]
        for res in result:
            if res["rootSource"] in eng_sabs:
                return f"{name}: " + translator.translate_text(text=res["value"], source_lang="EN",
                                                 target_lang="DE").text
        return f"{name}: " + translator.translate_text(text=response.json()["result"][0]["value"],
                                         source_lang=None,
                                         target_lang="DE").text
    else:
        return name


In [58]:
for annotation in tqdm(all_annotations):
    annotation["description"] = get_description(cui=annotation["cui"])

100%|██████████| 875/875 [00:00<00:00, 15233.59it/s]


In [59]:
for annotation in tqdm(all_annotations):
    if annotation["description"] is None:
        print(annotation)

100%|██████████| 875/875 [00:00<00:00, 1816839.60it/s]

{'annotation': Annotation(tech_term=None, lay_term='Langzeitzuckerwert', type=<TermType.LAY: 'LAY'>, span_start=1054, span_end=1072, synonyms=['HbA1c - Glykohämoglobin, Der Spiegel im Blut gibt Auskunft über die Blutzuckerwerte der letzten vier bis zwölf Wochen', 'HbA1c'], id=4619), 'cui': 'C0373638', 'description': None}
{'annotation': Annotation(tech_term=None, lay_term='Nierenschädlichkeit', type=<TermType.LAY: 'LAY'>, span_start=36, span_end=55, synonyms=['Nephrotoxizität', 'Nephrotoxizität'], id=1949), 'cui': 'C0599918', 'description': None}
{'annotation': Annotation(tech_term='Bandscheibenvorfall', lay_term=None, type=<TermType.TECH: 'TECH'>, span_start=107, span_end=126, synonyms=['Bandscheibenprolaps', 'Bandscheibenprolaps'], id=3931), 'cui': 'C0242362', 'description': None}



100%|██████████| 875/875 [00:00<00:00, 1171406.32it/s]

{'annotation': Annotation(tech_term=None, lay_term='Langzeitzuckerwert', type=<TermType.LAY: 'LAY'>, span_start=1054, span_end=1072, synonyms=['HbA1c - Glykohämoglobin, Der Spiegel im Blut gibt Auskunft über die Blutzuckerwerte der letzten vier bis zwölf Wochen', 'HbA1c'], id=4619), 'cui': 'C0373638', 'description': None}
{'annotation': Annotation(tech_term=None, lay_term='Nierenschädlichkeit', type=<TermType.LAY: 'LAY'>, span_start=36, span_end=55, synonyms=['Nephrotoxizität', 'Nephrotoxizität'], id=1949), 'cui': 'C0599918', 'description': None}
{'annotation': Annotation(tech_term='Bandscheibenvorfall', lay_term=None, type=<TermType.TECH: 'TECH'>, span_start=107, span_end=126, synonyms=['Bandscheibenprolaps', 'Bandscheibenprolaps'], id=3931), 'cui': 'C0242362', 'description': None}


In [60]:
no_desc_ann = [ann for ann in all_annotations if ann["description"] is None]
print(len(no_desc_ann))
all_annotations_with_desc = [ann for ann in all_annotations if ann["description"] is not None]

3
3


In [61]:
# parse annotations to prodigy samples
from data_loading import annotations_cache


In [71]:
prodigy_samples = []
for ann_with_cui_and_desc in tqdm(all_annotations_with_desc):
    annotation = ann_with_cui_and_desc["annotation"]
    cui = ann_with_cui_and_desc["cui"]
    description = ann_with_cui_and_desc["description"]
    spans = [ProdigyNERLabel(start=annotation.span_start, end=annotation.span_end)]
    text = sample_collection.get_sample_by_annotation_id(annotation.id).text
    # <a href="url">link text</a>

    html = f'<b>{cui}</b>: ' + description
    annotations_ids = get_annotation_ids(annotation.get_mention())
    prodigy_samples.append(
        ProdigySample(text=text, spans=spans, id=annotation.id, cui=cui, html=html,
                      annotation_ids=annotations_ids, meta=dict(url=f"https://uts.nlm.nih.gov/uts/umls/concept/{cui}")))
    

100%|██████████| 872/872 [00:00<00:00, 4344.51it/s]


In [72]:
with open(TLCPaths.project_data_path.joinpath("prodigy_samples_strict.jsonl"), "w") as fp:
    for annotation in prodigy_samples:
        fp.write(annotation.json(ensure_ascii=False) + "\n")

In [70]:
annotation.json()

'{"text": "Thread: [Immer morgens Durchfall]\\nText: [mmh, das kann bei dir ja wirklich gut an dem ganzen stress liegen.. bin ja jetzt kein arzt, aber vielleicht hat sich bei der blinddarmentfernung doch irgendwas entz\\u00fcndet und deshalb dort auch die schmerzen? k\\u00f6nnte ja sein, so ohne hintergrundwissen. ich find ne \\u00e4rztliche abkl\\u00e4rung jedenfalls auch sinnvoll. wenn man dann wei\\u00df, dass es nix schlimmes ist, dann kann man auch sicher andere wege finden, damit umzugehen. und so\'n examen belastet ja ganz klar, kenn ich ja \\nselber hab ich keine anderen beschwerden. ab und an links unten schmerzen, aber wirklich nur ab und an. ja und einen grummeligen verdauungstrakt, das ist sehr unangenehm, weil mich leute drauf ansprechen, wenn wir irgendwo sitzen und mein bauch wieder anf\\u00e4ngt.. aber das ist auch nicht immer..]\\n", "spans": [{"start": 166, "end": 185, "label": "Mention"}], "html": "<b>C0003614</a>: Blinddarmneubildungen: Tumore oder Krebs der APPENDI